In [669]:
#import libraries
#Need to pip install selenium
from splinter import Browser
from bs4 import BeautifulSoup as soup
import re
import pandas as pd
import matplotlib.pyplot as plt
import time
from datetime import date
from selenium.webdriver.common.keys import Keys

In [670]:
# Set up splinter
browser = Browser('chrome')

# base url
base_url = "https://www.facebook.com/marketplace/sanfrancisco/search?"

# search parameters
# days_listed = 7
deliveryMethod = "local_pick_up"
query = "honda%20insight"
availability = "in%20stock"

#full url
url = f"{base_url}availability={availability}&deliveryMethod={deliveryMethod}&query={query}"

#City to Test
CITY = "Raleigh, North Carolina"

In [671]:
#visit site
browser.visit(url)

In [672]:
if browser.is_element_present_by_css('div[aria-label="Close"]', wait_time=10):
    # Click on the element once it's found
    browser.find_by_css('div[aria-label="Close"]').first.click()

In [673]:
browser.find_by_text(f'San Francisco, California').click()

In [674]:
browser.find_by_css('input[aria-label="Location"]').click()
while (len(browser.find_by_css('input[aria-label="Location"]').value)) > 0:
    browser.find_by_css('input[aria-label="Location"]').type(Keys.BACKSPACE)
browser.find_by_css('input[aria-label="Location"]').type(CITY)

'Raleigh, North Carolina'

In [675]:
browser.find_by_css('input[aria-label="Location"]').type(Keys.DOWN)
browser.find_by_css('input[aria-label="Location"]').type(Keys.ENTER)

'\ue007'

In [676]:
browser.find_by_css('div[aria-label="Apply"]').click()


In [677]:
# scroll down to load more results

scroll_count = 4

scroll_delay = 2

for _ in range(scroll_count):
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    time.sleep(scroll_delay)

In [685]:
# Parse HTML
html = browser.html

#create BS object from HTML
market_soup = soup(html, "html.parser")

In [524]:
# End browsing session
browser.quit()

In [686]:
# Extract all the info, put into lists
titles_div = market_soup.find_all('span', class_="x1lliihq x6ikm8r x10wlt62 x1n2onr6")
titles_list = [title.text.strip() for title in titles_div]
titles_list.pop(0)
titles_list.pop(0)

prices_div = market_soup.find_all('span', class_="x193iq5w xeuugli x13faqbe x1vvkbs xlh3980 xvmahel x1n0sxbx x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x4zkp8e x3x7a5m x1lkfr7t x1lbecb7 x1s688f xzsf02u")
prices_list = [price.text.strip() for price in prices_div]

miles_div = market_soup.find_all('span', class_="x1lliihq x6ikm8r x10wlt62 x1n2onr6 xlyipyv xuxw1ft")
miles_list = [mile.text.strip() for mile in miles_div]

image_elems = market_soup.find_all('img', class_= "xt7dq6l xl1xv1r x6ikm8r x10wlt62 xh8yej3")
image_list = [image.get('src') for image in image_elems]

urls_elems = market_soup.find_all('a', class_= "x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1heor9g x1lku1pv")
urls_list = [url.get('href') for url in urls_elems]




In [687]:
titles_list

['2002 Honda insight Hybrid Hatchback 2D',
 '2019 Honda insight EX Sedan 4D',
 '2010 Honda insight EX Hatchback 4D',
 '2011 Honda insight Hybrid',
 '2012 Honda insight',
 '2019 Honda insight Touring Sedan 4D',
 '2019 Honda insight EX Sedan 4D',
 '2010 Honda insight EX Hatchback 4D',
 '2010 Honda insight LX Hatchback 4D',
 '2010 Honda insight EX',
 '2010 Honda insight EX Hatchback 4D',
 '2021 Honda insight EX Sedan 4D',
 '2014 Honda insight',
 '2012 Honda insight Hatchback 4D',
 '2019 Honda insight LX Sedan 4D',
 '2019 Honda insight LX Sedan 4D',
 '2010 Honda insight LX Hatchback 4D',
 '2011 Honda cr-z EX Coupe 2D',
 '2018 Honda civic EX-T Sedan 4D',
 '2019 Honda clarity Sliver',
 '2013 Honda civic Hybrid Sedan 4D',
 'Honda civic hybrid 2010',
 '2010-14 Honda Insight Hybrid Battery with modules and harness',
 '2019 Honda insight EX Sedan 4D',
 '2000 Honda insight Hybrid Hatchback 2D',
 '2012 Honda crosstour EX-L Sport Utility 4D',
 '2011 Honda insight Hatchback 4D',
 '2020 Honda insight

In [688]:
image_list

['https://scontent-lax3-2.xx.fbcdn.net/v/t39.30808-6/433140773_122115755444240073_5556977948482205665_n.jpg?stp=c0.86.828.828a_dst-jpg_s261x260&_nc_cat=100&ccb=1-7&_nc_sid=5f2048&_nc_ohc=oA_kO9bkxMkAb5PrWPc&_nc_ht=scontent-lax3-2.xx&oh=00_AfABWDBO_l8RT8b3j3enQppB3LgSjkA8TZKOo6JxOXaNNQ&oe=661CE5CF',
 'https://scontent-lax3-1.xx.fbcdn.net/v/t39.30808-6/434649668_1627156444777107_4400252646678218198_n.jpg?stp=c10.0.260.260a_dst-jpg_p261x260&_nc_cat=102&ccb=1-7&_nc_sid=5f2048&_nc_ohc=H3aO4t-zLAsAb54haNg&_nc_ht=scontent-lax3-1.xx&oh=00_AfAvHUqkgTE73D1uAKndK5XLHELgVPg2KgrMOGTesQDDOw&oe=661CE48C',
 'https://scontent-lax3-1.xx.fbcdn.net/v/t39.30808-6/434468414_8246897185327571_9060266950274269438_n.jpg?stp=c0.438.1080.1080a_dst-jpg_s261x260&_nc_cat=110&ccb=1-7&_nc_sid=5f2048&_nc_ohc=3j7PwDoYIFUAb4Vh6Xr&_nc_ht=scontent-lax3-1.xx&oh=00_AfBNqvP57gXpwKvND8LreYDdau80j12wE6Z3G3nnb6YClw&oe=661CD687',
 'https://scontent-lax3-2.xx.fbcdn.net/v/t39.30808-6/420223735_1626916268120174_7971832794913011992_n

In [689]:
miles_list

['Log In',
 'Log In',
 'Notify Me',
 'Create new listing',
 'Sort by',
 'Delivery method: Local pickup',
 'Condition',
 'Date listed',
 'Availability',
 'San Francisco · 40 mi',
 'Filters',
 'Notify Me',
 'San Francisco, CA',
 '165K miles',
 'Denair, CA',
 '150K miles',
 'Vallejo, CA',
 '125K miles',
 'Fresno, CA',
 '98K miles',
 'Roseville, CA',
 '134K miles',
 'San Jose, CA',
 '90K miles',
 'Sacramento, CA',
 '120K miles',
 'Burlingame, CA',
 '230K miles',
 'San Francisco, CA',
 '184K miles',
 'Sacramento, CA',
 '140K miles',
 'San Jose, CA',
 '113K miles',
 'Sacramento, CA',
 '149K miles · Dealership',
 'Sacramento, CA',
 '224K miles',
 'Modesto, CA',
 '108K miles',
 'Sausalito, CA',
 '170K miles',
 'Antioch, CA',
 '46K miles',
 'Petaluma, CA',
 '215K miles',
 'Brisbane, CA',
 '117K miles',
 'Pinole, CA',
 '70K miles',
 'Hayward, CA',
 '37K miles',
 'Castro Valley, CA',
 '124K miles',
 'Stockton, CA',
 'Hayward, CA',
 'Sacramento, CA',
 '79K miles',
 'Monterey, CA',
 '196K miles',
 

In [690]:
urls_list

['/marketplace/item/1492382191345101/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/1826502207823881/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/1886376908472499/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/382373454677975/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/25194297393550839/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/973014317516704/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/1419644868944637/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/1067662897667447/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/425378283505584/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/1168227390

In [691]:
prices_list

['$3,650',
 '$11,000',
 '$1,500',
 '$6,000',
 '$6,000',
 '$2,500',
 '$13,500',
 '$4,500',
 '$6,200',
 '·',
 '$8,000',
 '$7,940',
 '$12,950',
 '$6,000',
 '$6,900',
 '$13,500',
 '$15,950',
 '$3,650',
 '$8,000',
 '$17,500',
 '$13,950',
 '$9,000',
 '$3,500',
 '$450',
 '$17,800',
 '$4,000',
 '$2,000',
 '$6,750',
 '$17,300',
 '$7,200',
 '$8,500',
 '$100',
 '$4,050',
 '$555',
 '$13,500',
 '$5,700',
 '$18,300',
 '$6,500',
 '$75',
 '$6,000',
 '$100',
 '$7,500',
 '$65',
 '$3,500',
 '$19,800',
 '$500',
 '$22,000',
 '$550',
 '$3,900',
 '$4,500',
 '$21,499',
 '$2,500',
 '$5,100',
 '$13,500',
 '$3,500',
 '$555',
 '$11,900',
 '$20,499',
 '$4,500',
 '$7,900',
 '$18,000',
 '$555',
 '$9,900',
 '$4,000',
 '$11,000',
 '$450',
 '$2,500',
 '$350',
 '$11,888',
 '$19,500',
 '$2,000',
 '$400',
 '$8,500',
 '$7,950',
 '$99',
 '$5,900',
 '$500',
 '$5,000',
 '$1,500',
 '$20,000',
 '$1,000',
 '$7,500',
 '$3,750',
 '$8,000',
 '$2,999',
 '$2,500',
 '₹23,500',
 '$2,200',
 '$50',
 '$3,950',
 '$4,500',
 '$100',
 '$1',
 

In [692]:
miles_list

['Log In',
 'Log In',
 'Notify Me',
 'Create new listing',
 'Sort by',
 'Delivery method: Local pickup',
 'Condition',
 'Date listed',
 'Availability',
 'San Francisco · 40 mi',
 'Filters',
 'Notify Me',
 'San Francisco, CA',
 '165K miles',
 'Denair, CA',
 '150K miles',
 'Vallejo, CA',
 '125K miles',
 'Fresno, CA',
 '98K miles',
 'Roseville, CA',
 '134K miles',
 'San Jose, CA',
 '90K miles',
 'Sacramento, CA',
 '120K miles',
 'Burlingame, CA',
 '230K miles',
 'San Francisco, CA',
 '184K miles',
 'Sacramento, CA',
 '140K miles',
 'San Jose, CA',
 '113K miles',
 'Sacramento, CA',
 '149K miles · Dealership',
 'Sacramento, CA',
 '224K miles',
 'Modesto, CA',
 '108K miles',
 'Sausalito, CA',
 '170K miles',
 'Antioch, CA',
 '46K miles',
 'Petaluma, CA',
 '215K miles',
 'Brisbane, CA',
 '117K miles',
 'Pinole, CA',
 '70K miles',
 'Hayward, CA',
 '37K miles',
 'Castro Valley, CA',
 '124K miles',
 'Stockton, CA',
 'Hayward, CA',
 'Sacramento, CA',
 '79K miles',
 'Monterey, CA',
 '196K miles',
 

In [693]:
# pattern for City, State (ex: Los Angeles, CA)
location_pattern = re.compile(r"[a-zA-Z -']+[,]\s[A-Z][A-Z]")
# pattern for miles (ex: 100K miles)
miles_pattern = re.compile(r'([0-9.]+)[K]? miles')

CITY = 'Jackson'
STATE = 'MS'

# initialize empty list
miles_list2 = []

#iterate through the original mileage entries
for item in miles_list:

    # Current item is location, previous item is location (2 locations in a row)
    if location_pattern.match(item) and len(miles_list2) >= 1 and location_pattern.match(miles_list2[-1]):
        miles_list2.append('0K miles')
        miles_list2.append(item)

    # Current item is empty string, Previous item is location (Miles is empty string)
    elif len(item) == 0 and location_pattern.match(miles_list2[-1]) and len(miles_list2) >=1:
        miles_list2.append('0K miles')

    # Previous item is miles, and current item is miles (Missing Location)
    elif miles_pattern.match(item) and len(miles_list2) >= 1 and miles_pattern.match(miles_list2[-1]):
        miles_list2.append(f'{CITY}, {STATE}')
        miles_list2.append(item)

    # Previous item is location, and current item does not match (2 locations in a row)
    elif len(miles_list2) >= 1 and location_pattern.match(miles_list2[-1]) and miles_pattern.match(item) == None:
        miles_list2.append('0K miles')


    else:
        miles_list2.append(item)



In [694]:
miles_list2

['Log In',
 'Log In',
 'Notify Me',
 'Create new listing',
 'Sort by',
 'Delivery method: Local pickup',
 'Condition',
 'Date listed',
 'Availability',
 'San Francisco · 40 mi',
 'Filters',
 'Notify Me',
 'San Francisco, CA',
 '165K miles',
 'Denair, CA',
 '150K miles',
 'Vallejo, CA',
 '125K miles',
 'Fresno, CA',
 '98K miles',
 'Roseville, CA',
 '134K miles',
 'San Jose, CA',
 '90K miles',
 'Sacramento, CA',
 '120K miles',
 'Burlingame, CA',
 '230K miles',
 'San Francisco, CA',
 '184K miles',
 'Sacramento, CA',
 '140K miles',
 'San Jose, CA',
 '113K miles',
 'Sacramento, CA',
 '149K miles · Dealership',
 'Sacramento, CA',
 '224K miles',
 'Modesto, CA',
 '108K miles',
 'Sausalito, CA',
 '170K miles',
 'Antioch, CA',
 '46K miles',
 'Petaluma, CA',
 '215K miles',
 'Brisbane, CA',
 '117K miles',
 'Pinole, CA',
 '70K miles',
 'Hayward, CA',
 '37K miles',
 'Castro Valley, CA',
 '124K miles',
 'Stockton, CA',
 '0K miles',
 'Hayward, CA',
 '0K miles',
 'Sacramento, CA',
 '79K miles',
 'Monte

In [695]:
# Clean Location and Miles Data
miles_pattern_miles = r'([0-9.]+)[K]? miles'
miles_pattern_km = r'(\d+)K km'
location_pattern = r'[a-zA-Z ]+[,]\s[A-Z][A-Z]'

city_name = CITY.split(", ")[0]

full_city_pattern = r'[a-zA-Z ]+[,]\s[' + city_name + r']{' + f'{len(city_name)}' + r'}'


miles_clean = []
locations_clean =[]

for item in miles_list2:

    match_miles_km = re.search(miles_pattern_km, item)

    match_mileage_miles = re.search(miles_pattern_miles, item)

    match_location = re.search(location_pattern, item)

    match_full_city = re.search(full_city_pattern, item)


    if match_miles_km or match_mileage_miles or match_location or match_full_city:
        if match_miles_km:
            miles_clean.append(int(match_miles_km.group(1)) *1000)

        if match_mileage_miles:
            miles_clean.append(int(float(match_mileage_miles.group(1))) *1000)
            # print(match_mileage_miles)
            # print(int(float(match_mileage_miles.group(1))) *1000)

        if match_location:
            locations_clean.append(item)
            print(match_location)
            print(item)

        if match_full_city:
            locations_clean.append(item)

    else:
        print('NON-MATCHING MILES/LOCATION', item)

NON-MATCHING MILES/LOCATION Log In
NON-MATCHING MILES/LOCATION Log In
NON-MATCHING MILES/LOCATION Notify Me
NON-MATCHING MILES/LOCATION Create new listing
NON-MATCHING MILES/LOCATION Sort by
NON-MATCHING MILES/LOCATION Delivery method: Local pickup
NON-MATCHING MILES/LOCATION Condition
NON-MATCHING MILES/LOCATION Date listed
NON-MATCHING MILES/LOCATION Availability
NON-MATCHING MILES/LOCATION San Francisco · 40 mi
NON-MATCHING MILES/LOCATION Filters
NON-MATCHING MILES/LOCATION Notify Me
<re.Match object; span=(0, 17), match='San Francisco, CA'>
San Francisco, CA
<re.Match object; span=(0, 10), match='Denair, CA'>
Denair, CA
<re.Match object; span=(0, 11), match='Vallejo, CA'>
Vallejo, CA
<re.Match object; span=(0, 10), match='Fresno, CA'>
Fresno, CA
<re.Match object; span=(0, 13), match='Roseville, CA'>
Roseville, CA
<re.Match object; span=(0, 12), match='San Jose, CA'>
San Jose, CA
<re.Match object; span=(0, 14), match='Sacramento, CA'>
Sacramento, CA
<re.Match object; span=(0, 14), m

In [696]:
len(locations_clean)

96

In [697]:
len(miles_clean)

96

In [698]:
# Make Prices into Integer
prices_clean = []

for price in prices_list:
    if price == 'Free':
        prices_clean.append(0)
    elif price == "·":
        prices_clean.append(0)
    else:
        prices_clean.append(int(re.sub(r'[₹,M,X,$,]','', price)))


In [699]:
# Make URLS full url
urls_clean = []

for url in urls_list:
    urls_clean.append('https://www.facebook.com' + url)


In [700]:
len(titles_list)

96

In [701]:
# add all values to a list of dictionaries
vehicles_list = []

for i, item in enumerate(titles_list):
    cars_dict = {}
    first_gen = False
    insight = False
    parts = False

    year = 0
    year_pattern = r'[0-9]{4}'
    year_match = re.search(year_pattern,titles_list[i])
    first_gen_years = [2000, 2001, 2002, 2003, 2004, 2005, 2006]

    #Checks for year
    if year_match:
        year = int(year_match[0])
        print(year_match)
        print(year)

    #Checks if insight is actually in the title
    if "insight" in titles_list[i].lower():
        insight = True

    #Checks if any cars are 2000 - 2006
    if year_match:
        if any(x == int(year_match[0]) for x in first_gen_years) and insight:
            print("first gen exists")
            first_gen = True

    #Checks if car listing is for parts
    if "part" in titles_list[i].lower():
        parts = True


    #Splits up the City and State from location
    city = locations_clean[i].split(', ')[0]
    state = locations_clean[i].split(', ')[-1]



    # Map out key value pairs
    cars_dict["date"] = date.today()
    cars_dict["title"] = titles_list[i]
    cars_dict["price"] = prices_clean[i]
    cars_dict["city"] = city
    cars_dict["state"] = state
    cars_dict["mileage"] = miles_clean[i]
    cars_dict["url"] = urls_clean[i]
    cars_dict["image"] = image_list[i]
    cars_dict["insight"] = insight
    cars_dict["first_gen"] = first_gen
    cars_dict["parts"] = parts
    cars_dict["site"] = "fb"
    cars_dict["year"] = year
    vehicles_list.append(cars_dict)


<re.Match object; span=(0, 4), match='2002'>
2002
first gen exists
<re.Match object; span=(0, 4), match='2019'>
2019
<re.Match object; span=(0, 4), match='2010'>
2010
<re.Match object; span=(0, 4), match='2011'>
2011
<re.Match object; span=(0, 4), match='2012'>
2012
<re.Match object; span=(0, 4), match='2019'>
2019
<re.Match object; span=(0, 4), match='2019'>
2019
<re.Match object; span=(0, 4), match='2010'>
2010
<re.Match object; span=(0, 4), match='2010'>
2010
<re.Match object; span=(0, 4), match='2010'>
2010
<re.Match object; span=(0, 4), match='2010'>
2010
<re.Match object; span=(0, 4), match='2021'>
2021
<re.Match object; span=(0, 4), match='2014'>
2014
<re.Match object; span=(0, 4), match='2012'>
2012
<re.Match object; span=(0, 4), match='2019'>
2019
<re.Match object; span=(0, 4), match='2019'>
2019
<re.Match object; span=(0, 4), match='2010'>
2010
<re.Match object; span=(0, 4), match='2011'>
2011
<re.Match object; span=(0, 4), match='2018'>
2018
<re.Match object; span=(0, 4), ma

In [702]:
vehicles_list

[{'date': datetime.date(2024, 4, 10),
  'title': '2002 Honda insight Hybrid Hatchback 2D',
  'price': 3650,
  'city': 'San Francisco',
  'state': 'CA',
  'mileage': 165000,
  'url': 'https://www.facebook.com/marketplace/item/1492382191345101/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
  'image': 'https://scontent-lax3-2.xx.fbcdn.net/v/t39.30808-6/433140773_122115755444240073_5556977948482205665_n.jpg?stp=c0.86.828.828a_dst-jpg_s261x260&_nc_cat=100&ccb=1-7&_nc_sid=5f2048&_nc_ohc=oA_kO9bkxMkAb5PrWPc&_nc_ht=scontent-lax3-2.xx&oh=00_AfABWDBO_l8RT8b3j3enQppB3LgSjkA8TZKOo6JxOXaNNQ&oe=661CE5CF',
  'insight': True,
  'first_gen': True,
  'parts': False,
  'site': 'fb',
  'year': 2002},
 {'date': datetime.date(2024, 4, 10),
  'title': '2019 Honda insight EX Sedan 4D',
  'price': 11000,
  'city': 'Denair',
  'state': 'CA',
  'mileage': 150000,
  'url': 'https://www.facebook.com/marketplace/item/1826502207823881/?ref=search&referral_code=null&referral_story_type=post&__

In [703]:
vehicles_df = pd.DataFrame(vehicles_list)

In [704]:
vehicles_df

,date,title,price,city,state,mileage,url,image,insight,first_gen,parts,site,year
0,2024-04-10,2002 Honda insight Hybrid Hatchback 2D,3650,San Francisco,CA,165000,https://www.facebook.com/marketplace/item/1492...,https://scontent-lax3-2.xx.fbcdn.net/v/t39.308...,True,True,False,fb,2002
1,2024-04-10,2019 Honda insight EX Sedan 4D,11000,Denair,CA,150000,https://www.facebook.com/marketplace/item/1826...,https://scontent-lax3-1.xx.fbcdn.net/v/t39.308...,True,False,False,fb,2019
2,2024-04-10,2010 Honda insight EX Hatchback 4D,1500,Vallejo,CA,125000,https://www.facebook.com/marketplace/item/1886...,https://scontent-lax3-1.xx.fbcdn.net/v/t39.308...,True,False,False,fb,2010
3,2024-04-10,2011 Honda insight Hybrid,6000,Fresno,CA,98000,https://www.facebook.com/marketplace/item/3823...,https://scontent-lax3-2.xx.fbcdn.net/v/t39.308...,True,False,False,fb,2011
4,2024-04-10,2012 Honda insight,6000,Roseville,CA,134000,https://www.facebook.com/marketplace/item/2519...,https://scontent-lax3-2.xx.fbcdn.net/v/t39.308...,True,False,False,fb,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,2024-04-10,Parting Out 2010 Honda Insight,100,Rancho Cordova,CA,0,https://www.facebook.com/marketplace/item/1877...,https://scontent-lax3-1.xx.fbcdn.net/v/t45.532...,True,False,True,fb,2010
92,2024-04-10,2019 Honda civic EX Hatchback 4D,1,Rancho Cordova,CA,63000,https://www.facebook.com/marketplace/item/4267...,https://scontent-lax3-2.xx.fbcdn.net/v/t39.308...,False,False,False,fb,2019
93,2024-04-10,2009 Toyota prius,12500,Richmond,CA,180000,https://www.facebook.com/marketplace/item/2923...,https://scontent-lax3-1.xx.fbcdn.net/v/t45.532...,False,False,False,fb,2009
94,2024-04-10,2007 Honda civic Hybrid Sedan 4D,900,Sonoma,CA,190000,https://www.facebook.com/marketplace/item/7797...,https://scontent-lax3-1.xx.fbcdn.net/v/t39.308...,False,False,False,fb,2007


In [705]:
filtered_df = vehicles_df[vehicles_df['insight']==True]

In [706]:
filtered_df

,date,title,price,city,state,mileage,url,image,insight,first_gen,parts,site,year
0,2024-04-10,2002 Honda insight Hybrid Hatchback 2D,3650,San Francisco,CA,165000,https://www.facebook.com/marketplace/item/1492...,https://scontent-lax3-2.xx.fbcdn.net/v/t39.308...,True,True,False,fb,2002
1,2024-04-10,2019 Honda insight EX Sedan 4D,11000,Denair,CA,150000,https://www.facebook.com/marketplace/item/1826...,https://scontent-lax3-1.xx.fbcdn.net/v/t39.308...,True,False,False,fb,2019
2,2024-04-10,2010 Honda insight EX Hatchback 4D,1500,Vallejo,CA,125000,https://www.facebook.com/marketplace/item/1886...,https://scontent-lax3-1.xx.fbcdn.net/v/t39.308...,True,False,False,fb,2010
3,2024-04-10,2011 Honda insight Hybrid,6000,Fresno,CA,98000,https://www.facebook.com/marketplace/item/3823...,https://scontent-lax3-2.xx.fbcdn.net/v/t39.308...,True,False,False,fb,2011
4,2024-04-10,2012 Honda insight,6000,Roseville,CA,134000,https://www.facebook.com/marketplace/item/2519...,https://scontent-lax3-2.xx.fbcdn.net/v/t39.308...,True,False,False,fb,2012
5,2024-04-10,2019 Honda insight Touring Sedan 4D,2500,San Jose,CA,90000,https://www.facebook.com/marketplace/item/9730...,https://scontent-lax3-1.xx.fbcdn.net/v/t39.308...,True,False,False,fb,2019
6,2024-04-10,2019 Honda insight EX Sedan 4D,13500,Sacramento,CA,120000,https://www.facebook.com/marketplace/item/1419...,https://scontent-lax3-1.xx.fbcdn.net/v/t39.308...,True,False,False,fb,2019
7,2024-04-10,2010 Honda insight EX Hatchback 4D,4500,Burlingame,CA,230000,https://www.facebook.com/marketplace/item/1067...,https://scontent-lax3-1.xx.fbcdn.net/v/t39.308...,True,False,False,fb,2010
8,2024-04-10,2010 Honda insight LX Hatchback 4D,6200,San Francisco,CA,184000,https://www.facebook.com/marketplace/item/4253...,https://scontent-lax3-1.xx.fbcdn.net/v/t39.308...,True,False,False,fb,2010
9,2024-04-10,2010 Honda insight EX,Pending,Sacramento,CA,140000,https://www.facebook.com/marketplace/item/1168...,https://scontent-lax3-1.xx.fbcdn.net/v/t39.308...,True,False,False,fb,2010


In [328]:
csv_file_path = f'/Users/evanishibashi/Projects/insight/csv/fb/{date.today()}.csv'

filtered_df.to_csv(csv_file_path, index=False)